In [ ]:
import datetime
from email.message import EmailMessage
import shutil
import smtplib
import ssl
import sys
import zipfile

from email.mime.text import MIMEText
from email.mime.multipart import MIMEMultipart
from email.mime.base import MIMEBase
from email.mime.application import MIMEApplication
from email import encoders
import os.path

os.chdir(r'D:\algo2\IBridgePy_Win_Anaconda38_64 - Copy')
sys.path.append(os.path.join(os.getcwd(), '..'))

from trader_factory import build_active_IBridgePy_plus
from IBridgePy.IbridgepyTools import symbol

from Utils import Algos
from Utils import EnterExits
from Utils import IndicatorCalculations
from Utils import Orders
from Utils import Plotters
from Utils import Utils
from UnitTests import testutils


In [ ]:
def sendEmailEndOfDay(email_sender:str = "lanecapitalgroup.feeney@gmail.com",
                      email_receiver:list = ["richfeeney6@gmail.com"],#"lanecapitalgroup1@gmail.com"],
                      attachments: list = [r"D:\algo2\IBridgePy_Win_Anaconda38_64 - Copy\Output\candlestick_"+str(datetime.date.today())+".zip"]):

    
    email_password = "bhpoxfksvoubpaxl"
    subject = "Daily Plots "+str(datetime.date.today())
    body = """See attached for plots and data for """+str(datetime.date.today())
    
  
    for email in email_receiver:
        msg = MIMEMultipart()
        msg['From'] = email_sender
        msg['To'] = email
        msg['Subject'] = subject

        msg.attach(MIMEText(body, 'plain'))

        # Setup the attachment
        filename = os.path.basename(attachments[0])
        attachment = open(attachments[0], "rb")
        part = MIMEBase('application', 'octet-stream')
        part.set_payload(attachment.read())
        encoders.encode_base64(part)
        part.add_header('Content-Disposition', "attachment; filename= %s" % filename)

        # Attach the attachment to the MIMEMultipart object
        for a in attachments:
            with open(a,'rb')as f:
                msg.attach(MIMEApplication(f.read(), Name=a.split("\\")[-1]))

        server = smtplib.SMTP('smtp.gmail.com', 587)
        server.starttls()
        server.login(email_sender, email_password)
        text = msg.as_string()
        server.sendmail(email_sender, email, text)
        server.quit()
    


In [ ]:
def ZipFilesForDelivery():
    """ This function runs at the end of the day and copies output logs to a predetermined directory and zips the files """
    
    #make directory to pass folders
    zipPath = r"D:\algo2\IBridgePy_Win_Anaconda38_64 - Copy\Output\candlestick_"+str(datetime.date.today())
    if not os.path.isdir(zipPath):
        os.mkdir(zipPath)

    # pass daily log, price data, and trader log
    logPath = r"D:\algo2\IBridgePy_Win_Anaconda38_64 - Copy\Log\Daily_Log_"+str(datetime.date.today())+".csv"
    pricePath = r"D:\algo2\IBridgePy_Win_Anaconda38_64 - Copy\Log\Daily_Price_Data_"+str(datetime.date.today())+".csv"
    traderPath = r"D:\algo2\IBridgePy_Win_Anaconda38_64 - Copy\Log\TraderLog_"+str(datetime.date.today())+".txt"

    # copy files to new directory
    shutil.copy(logPath,zipPath+"\\"+logPath.split("\\")[-1])
    shutil.copy(pricePath,zipPath+"\\"+pricePath.split("\\")[-1])
    shutil.copy(traderPath,zipPath+"\\"+traderPath.split("\\")[-1])

    # Create a zip folder
    zipName = zipPath+"\\..\\OutputData_"+str(datetime.date.today())+".zip"
    Utils.zipFolder(zipPath, zipName)

In [ ]:
def zipFolder(zipPath, zipName):
    with zipfile.ZipFile(zipName, 'w', zipfile.ZIP_DEFLATED) as zip_ref:
        for folder_name, subfolders, filenames in os.walk(zipPath):
            print(folder_name)
            for filename in filenames:
                print(filename)
                file_path = os.path.join(folder_name, filename)
                zip_ref.write(file_path, arcname=os.path.relpath(file_path, zipPath))

    zip_ref.close()